###### Generating A Preference Dataset With Llama 3.1 70B And Ollama
###### Preference finetuning is a process to align an instruction-finetuned LLM with human preferences
###### There are multiple ways to create a dataset for preference finetuning an LLM
###### We use the instruction-finetuned LLM to generate multiple responses and have humans rank them based on ###### their preference and/or given preference criteria
###### We use the instruction-finetuned LLM to generate multiple responses and have LLMs rank them based on ###### given preference criteria
###### We use an LLM to generate preferred and dispreferred responses given certain preference criteria
###### In this notebook, we consider approach 3
###### This notebook uses a 70-billion-parameter Llama 3.1-Instruct model through ollama to generate preference ###### labels for an instruction dataset
###### The expected format of the instruction dataset is as follows:
###### Input
###### 
###### [
######     {
######         "instruction": "What is the state capital of California?",
######         "input": "",
######         "output": "The state capital of California is Sacramento.",
######     },
######     {
######         "instruction": "Provide a synonym for 'fast'.",
######         "input": "",
######         "output": "A synonym for 'fast' is 'quick'.",
######     },
######     {
######         "instruction": "What is the capital of Greece?",
######         "input": "",
######         "output": "The capital of Greece is Athens.",
###### 
######     },
###### ...
###### ]
###### The output dataset will look as follows, where more polite responses are preferred ('chosen'), and more impolite responses are dispreferred ('rejected'):

###### [
######     {
######         "instruction": "What is the state capital of California?",
######         "input": "",
######         "output": "The state capital of California is Sacramento.",
######         "rejected": "Look, the state capital of California is obviously Sacramento.",
######         "chosen": "The state capital of California is Sacramento."
######    },
######    {
######        "instruction": "Provide a synonym for 'fast'.",
######        "input": "",
######        "output": "A synonym for 'fast' is 'quick'.",
######        "chosen": "A suitable alternative to 'fast' would be 'quick'.",
######        "rejected": "A synonym for 'fast' is 'quick'."
######    },
######    {
######         "instruction": "What is the capital of Greece?",
######         "input": "",
######         "output": "The capital of Greece is Athens.",
######         "chosen": "I'd be happy to help! The capital of Greece is indeed Athens.",
######         "rejected": "The capital of Greece is Athens."
######     },
###### ...
###### ]

#### Output

###### The code doesn't require a GPU and runs on a laptop given enough RAM

In [ ]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

#### Installing Ollama and Downloading Llama 3.1
###### Ollama is an application to run LLMs efficiently
###### It is a wrapper around llama.cpp, which implements LLMs in pure C/C++ to maximize efficiency
###### Note that it is a tool for using LLMs to generate text (inference), not training or finetuning LLMs
###### Prior to running the code below, install ollama by visiting https://ollama.com and following the ###### instructions (for instance, clicking on the "Download" button and downloading the ollama application for ###### your operating system)
###### For macOS and Windows users, click on the ollama application you downloaded; if it prompts you to install ###### the command line usage, say "yes"
###### 
###### Linux users can use the installation command provided on the ollama website
###### 
###### In general, before we can use ollama from the command line, we have to either start the ollama ###### application or run ollama serve in a separate terminal

###### With the ollama application or ollama serve running, in a different terminal, on the command line, execute the following command to try out the 70-billion-parameter Llama 3.1 model

# 70B model
###### ollama run llama3.1:70b
###### The output looks like as follows:
###### 
###### $ ollama run llama3.1:70b
###### pulling manifest
###### pulling aa81b541aae6... 100% ▕████████████████▏ 39 GB
###### pulling 8cf247399e57... 100% ▕████████████████▏ 1.7 KB
###### pulling f1cd752815fc... 100% ▕████████████████▏ 12 KB
###### pulling 56bb8bd477a5... 100% ▕████████████████▏ 96 B
###### pulling 3c1c2d3df5b3... 100% ▕████████████████▏ 486 B
###### verifying sha256 digest
###### writing manifest
###### removing any unused layers
###### success
###### Note that llama3.1:70b refers to the instruction finetuned 70-billion-parameter Llama 3.1 model

###### Alternatively, you can also use the smaller, more resource-effiicent 8-billion-parameters Llama 3.1 ###### model, by replacing llama3.1:70b with llama3.1

###### After the download has been completed, you will see a command line prompt that allows you to chat with ###### the model

###### Try a prompt like "What do llamas eat?", which should return an output similar to the following:

###### >>> What do llamas eat?
###### Llamas are ruminant animals, which means they have a four-chambered 
###### stomach and eat plants that are high in fiber. In the wild, llamas 
###### typically feed on:
###### 1. Grasses: They love to graze on various types of grasses, including tall 
###### grasses, wheat, oats, and barley.
###### You can end this session using the input /bye
###### Using Ollama's REST API
###### Now, an alternative way to interact with the model is via its REST API in Python via the following ###### function
###### Before you run the next cells in this notebook, make sure that ollama is still running, as described ###### above, via
###### ollama serve in a terminal
###### the ollama application
###### Next, run the following code cell to query the model
###### First, let's try the API with a simple example to make sure it works as intended:

In [ ]:
import urllib.request
import json


def query_model(prompt, model="llama3.1:70b", url="http://localhost:11434/api/chat"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

#### Load JSON Entries
###### Now, let's get to the data generation part
###### Here, for a hands-on example, we use the instruction-data.json file that we originally used to instruction-finetune the model in chapter 7:

In [ ]:
from pathlib import Path

json_file = Path("..", "01_main-chapter-code", "instruction-data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

###### The structure of this file is as follows, where we have the given response in the test dataset ('output') that we trained the model to generate via instruction finetuning based on the 'input' and 'instruction'


In [ ]:
json_data[0]

###### Below is a small utility function that formats the instruction and input:

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

###### Now, let's try the ollama API to generate a 'chosen' and 'rejected' response for preference tuning a model
###### Here, to for illustration purposes, we create answers that are more or less polite

In [ ]:
import random


for entry in json_data[:5]:
    
    politeness = random.choice(["polite", "impolite"])    
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"\n{politeness} response:")
    print(">>", query_model(prompt)) 

###### If we find that the generated responses above look reasonable, we can go to the next step and apply the prompt to the whole dataset
###### Here, we add a 'chosen' key for the preferred response and a 'rejected' response for the dispreferred response

In [ ]:
import random
from tqdm import tqdm

def generate_model_responses(json_data):

    for i, entry in enumerate(tqdm(json_data, desc="Writing entries")):
        politeness = random.choice(["polite", "impolite"])    
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"slightly rewrite the output to be more {politeness}."
            "Keep the modification minimal."
            "Only return return the generated response and nothing else."
        )
        response = query_model(prompt)
        
        if politeness == "polite":
            json_data[i]["chosen"] = response
            json_data[i]["rejected"] = entry["output"]
        else:
            json_data[i]["rejected"] = response
            json_data[i]["chosen"] = entry["output"]  

###### Let's now apply this evaluation to the whole dataset and compute the average score of each model (this takes about 1 minute per model on an M3 MacBook Air laptop)
###### Note that ollama is not fully deterministic across operating systems (as of this writing) so the numbers you are getting might slightly differ from the ones shown below

In [ ]:
generate_model_responses(json_data)

In [ ]:
with open("instruction-data-with-preference.json", "w") as file:
    json.dump(json_data, file, indent=4)